In [ ]:
import numpy as np
import threading
import pandas as pd

Eligible inorganic metal halide perovskite possible elements at various points

In [ ]:
A_cations = ['Cs','Rb']
#B_site is bivalent cationic homovalent doping
B_cations = ['Be','Mg','Ca','Sr','Ba','Mn','Fe','Co','Ni','Pd','Pt','Cu','Zn','Cd','Hg','Ge','Sn','Pb','Eu','Tm','Yb','V']
X_anions = ['I','Cl','Br']

#The ionic radius is determined by the coordination number
ion_radii = {
    'Cs': 1.88, 'Rb': 1.72, 'Cu': 0.73, 'Pb': 1.19, 'Eu': 1.17, 'Co': 0.74, 'Mn':0.83 ,'Pd': 0.86,'Cl':1.81,'Br':1.96,
    'Sn': 1.03, 'I': 2.2,'Zn':0.74,'Cd':0.95,'Be':0.45,'Mg':0.72,'Ca':1,'Sr':1.18,'Ba':1.35,'Fe':0.78,'Ni':0.7,'Pt':0.98,'Hg':1.03,'Ge':0.73,'Tm':1.03,'Yb':1.02,'V':0.79
}

Calculate the radius of each location of the virtual compound

In [ ]:
import concurrent.futures
from pymatgen.core import Composition
def generate_candidate(i,site):
    candidates = []
    if site == 'A':
        for X in X_anions:
            for A1 in A_cations: 
                for A2 in A_cations:
                    for B in B_cations:
                        B1, B2 = B,B
                        b = 1
                        a = i
                        if (A1 != A2) :
                            candidate = ''.join([X, '3', A1, str(a), A2, str(1 - a), B1, str(b), B2, str(1 - b)])
                            tmp_dict = {'formula': Composition(candidate).reduced_formula,
                                        'A1': A1, 'A1_ratio': a, 'A2': A2, 'A2_ratio': 1 - a,
                                        'B1': B1, 'B1_ratio': b, 'B2': B2, 'B2_ratio': 1 - b,
                                        'X': X, 'X_ratio': 3,
                                        'Ra': ion_radii[A1] * a + ion_radii[A2] * (1 - a),
                                        'Rb': ion_radii[B1] * b + ion_radii[B2] * (1 - b),
                                        'Rx': ion_radii[X]}
                            candidates.append(tmp_dict)
    elif site == 'B':
        for X in X_anions:
            for A in A_cations:
                for B1 in B_cations:
                    for B2 in B_cations:
                        A1, A2 = A,A
                        a = 1
                        b = i 
                        if (B1 != B2):
                            candidate = ''.join([X, '3', A1, str(a), A2, str(1 - a), B1, str(b), B2, str(1 - b)])
                            tmp_dict = {'formula': Composition(candidate).reduced_formula,
                                        'A1': A1, 'A1_ratio': a, 'A2': A2, 'A2_ratio': 1 - a,
                                        'B1': B1, 'B1_ratio': b, 'B2': B2, 'B2_ratio': 1 - b,
                                        'X': X, 'X_ratio': 3,
                                        'Ra': ion_radii[A1] * a + ion_radii[A2] * (1 - a),
                                        'Rb': ion_radii[B1] * b + ion_radii[B2] * (1 - b),
                                        'Rx': ion_radii[X]}
                            candidates.append(tmp_dict)
    elif site == 'X':
        for A in A_cations:
            for B in B_cations:
                for X1 in X_anions:
                    for X2 in X_anions:
                        A1,A2=A,A
                        B1,B2=B,B
                        a=1
                        b=1
                        x=i
                        if (X1 != X2):
                            candidate = ''.join([X1, str(x), X2, str(3- x), A1, str(a), A2, str(1 - a), B1, str(b), B2, str(1 - b)])
                            tmp_dict = {'formula': Composition(candidate).reduced_formula,
                                        'A1': A1, 'A1_ratio': a, 'A2': A2, 'A2_ratio': 1 - a,
                                        'B1': B1, 'B1_ratio': b, 'B2': B2, 'B2_ratio': 1 - b,
                                        'X1': X1, 'X1_ratio': x, 'X2': X2, 'X2_ratio': 3 - x,
                                        'Ra': ion_radii[A1] * a + ion_radii[A2] * (1 - a),
                                        'Rb': ion_radii[B1] * b + ion_radii[B2] * (1 - b),
                                        'Rx': ion_radii[X1] * x + ion_radii[X2] * (3 - x)}
                            candidates.append(tmp_dict)     
    return candidates

In [ ]:
def generate_candidates_concurrently(dope_range,site):
    all_candidates = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=128) as executor:
        # Create all possible combinations
        futures = [executor.submit(generate_candidate, a,site) 
                   for a in dope_range]
        for future in concurrent.futures.as_completed(futures):
            all_candidates.extend(future.result())
        result = pd.DataFrame(all_candidates)
    return result.drop_duplicates(subset=['formula'], keep='first').reset_index(drop=True)

In [ ]:
dope_range = np.arange(0, 1.01, 0.01)

In [ ]:
site = 'A'
result_a = generate_candidates_concurrently(dope_range,site)
result_a

In [ ]:
site = 'B'
result_b = generate_candidates_concurrently(dope_range,site)
result_b

In [ ]:
dope_range = np.arange(0, 3.01, 0.01)
site = 'X'
result_x = generate_candidates_concurrently(dope_range,site)
result_x

In [ ]:
df0 = pd.concat([result_a,result_b,result_x],axis = 0).reset_index(drop=True)

The new tolerance factor was calculated to screen the eligible hypothetical MHP materials

In [ ]:
import seaborn as sns

nA=1
df0['New_Tolerance_Factor'] = df0['Rx']/df0['Rb'] - nA*(nA - (df0['Ra']/df0['Rb'])/np.log1p(df0['Ra']/df0['Rb'] - 1))
df1 = df0[['formula', 'Ra', 'Rb', 'Rx', 'New_Tolerance_Factor']]
sns.distplot(df1['New_Tolerance_Factor'])

In [ ]:
df12 = df1.query('New_Tolerance_Factor<4.18')
sns.distplot(df12['New_Tolerance_Factor'])